In [48]:
import tensorflow as tf

import random
import json
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np

In [72]:
with open("intents.json") as file:
    data=json.load(file)
    

In [73]:
words=[]
labels=[]
docs_x=[]
docs_y=[]

In [74]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent["tag"] not in labels:
        labels.append(intent["tag"])         

In [75]:
words=[stemmer.stem(w.lower()) for w in words if w not in "?"]
words=sorted(list(set(words)))

In [76]:
labels=sorted(labels)
print(words)

['19', 'a', 'ag', 'am', 'anyon', 'ar', 'buy', 'cal', 'can', 'charg', 'cost', 'could', 'covid', 'covid-19', 'cre', 'cya', 'day', 'develop', 'do', 'due', 'eat', 'fee', 'from', 'funct', 'get', 'good', 'goodby', 'guy', 'hav', 'hello', 'hi', 'hour', 'how', 'i', 'id', 'is', 'lat', 'leav', 'lik', 'mad', 'menu', 'nam', 'of', 'old', 'on', 'op', 'origin', 'pandaem', 'pay', 'perform', 'reccommend', 'sav', 'savi', 'see', 'serv', 'should', 'someth', 'the', 'ther', 'to', 'up', 'us', 'vir', 'what', 'when', 'who', 'yo', 'you']


In [77]:
training=[]
output=[]
output_empty = [0 for _ in range(len(labels))]
for x, doc in enumerate(docs_x):
    bag = []
    wrds=[stemmer.stem(w.lower()) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    
    output_row = output_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)
    

In [78]:
training=np.array(training)
output=np.array(output)

In [79]:
import tflearn

In [80]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [81]:
net=tflearn.input_data(shape=[None,len(training[0])])
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net, len(output[0]), activation="softmax")
net=tflearn.regression(net)

In [82]:
model=tflearn.DNN(net)

In [83]:
model.fit(training,output, n_epoch=1000,batch_size=8,show_metric=True)

Training Step: 4999  | total loss: 0.01916 | time: 0.019s
| Adam | epoch: 1000 | loss: 0.01916 - acc: 0.9996 -- iter: 32/40
Training Step: 5000  | total loss: 0.01882 | time: 0.023s
| Adam | epoch: 1000 | loss: 0.01882 - acc: 0.9996 -- iter: 40/40
--


In [61]:
model.save("chatbotmodel")

INFO:tensorflow:D:\learn\ML\chatbot\chatbotmodel is not in all_model_checkpoint_paths. Manually adding it.


In [62]:
def bag_of_words(s,words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(w.lower()) for w in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    return(np.array(bag))

In [66]:
def chat():
    print("Lets start chatting! (Enter quit to close)")
    while True:
        
        inp=input("you: ")
        if(inp=="quit"):
            break
        result=model.predict([bag_of_words(inp,words)])[0]
        result_index=np.argmax(result)
        tag=labels[result_index]
        if(result[result_index]>0.8):
            for tg in data["intents"]:
                if (tg["tag"] == tag):
                    response = tg["responses"]
            print(random.choice(response))
        else:
            print("I didn't get that, try another question!")


chat()

In [67]:
chat()

Lets start chatting! (Enter quit to close)
you: hello
Hi there, how can I help?
you: what can you do
I am here to help!
you: what kind of help
I didn't get that, try another question!
you: what are your feautures
I didn't get that, try another question!
you: what can you do
I am here to help!
you: what can you operate
I am here to help!
you: what can you tell
I am here to help!
you: quit
